In [2]:
import pandas as pd
import numpy as np
import string
import joblib
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
## to download tool kit for data understanding process
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\420317\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\420317\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\420317\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Convert ratings to binary sentiment labels 

In [4]:
data = pd.read_csv(r"C:\Users\420317\Downloads\ICT-AI-ASSIGNMENT-main\Ast_code\drugLibTest_raw.tsv", sep='\t')

#### similar approch for 1st questions EDA

In [5]:
data.columns

Index(['Unnamed: 0', 'urlDrugName', 'rating', 'effectiveness', 'sideEffects',
       'condition', 'benefitsReview', 'sideEffectsReview', 'commentsReview'],
      dtype='object')

In [6]:
## condition apply for 7 is positive and else negative values

sentiments = []
for data_rate in data['rating']:
    if data_rate >= 7:
        sentiments.append(1)
    else :
        sentiments.append(0)
data['sentiment'] = sentiments

#### Preprocess text

In [7]:
data.columns

Index(['Unnamed: 0', 'urlDrugName', 'rating', 'effectiveness', 'sideEffects',
       'condition', 'benefitsReview', 'sideEffectsReview', 'commentsReview',
       'sentiment'],
      dtype='object')

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\420317\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import nltk
print(nltk.data.find('tokenizers')) 

C:\Users\420317\AppData\Roaming\nltk_data\tokenizers


In [10]:
nltk.data.path.append(r'C:\Users\420317\AppData\Roaming\nltk_data')
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\420317\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
stop_words = set(ENGLISH_STOP_WORDS)

def preprocess(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

data['cln_review'] = data['commentsReview'].apply(preprocess)

In [ ]:
vector_file = TfidfVectorizer(max_features=10000)
x = vector_file.fit_transform(data['cln_review'])
y = data['sentiment']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [17]:
models_and_predictions = {
    "LogisticRegression": {
        "model": linear_m,
        "predictions": linear_m.predict(X_test)
    },
    "RandomForestClassifier": {
        "model": random_f,
        "predictions": random_f.predict(X_test)
    }
}

for model_name, data in models_and_predictions.items():
    model = data["model"]
    predictions = data["predictions"]
    print(f"Model: {model_name}")
    print(classification_report(y_test, lr_preds))
    print("-" * 50)

Model: LogisticRegression
              precision    recall  f1-score   support

           0       0.40      0.03      0.05        73
           1       0.65      0.98      0.78       135

    accuracy                           0.64       208
   macro avg       0.53      0.50      0.42       208
weighted avg       0.56      0.64      0.52       208

--------------------------------------------------
Model: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.40      0.03      0.05        73
           1       0.65      0.98      0.78       135

    accuracy                           0.64       208
   macro avg       0.53      0.50      0.42       208
weighted avg       0.56      0.64      0.52       208

--------------------------------------------------


#### model build for tunning system

In [14]:
params = {
    'n_estimators': [100, 200],
    'max_depth': [ 10, 20],
    'min_samples_split': [2, 5,7],
}
grid = GridSearchCV(RandomForestClassifier(), param_grid=params, cv=3, scoring='f1', n_jobs=-1)
grid.fit(X_train, y_train)

print("Best parameter model:", grid.best_params_)
best_model = grid.best_estimator_

Best parameter model: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}


### save model

In [ ]:
joblib.dump(best_model, "sentiment_model_analysis.pkl")
joblib.dump(vector_file, "fidf_vectorizer_model.pkl")